In [1]:
import numpy as np
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from tqdm import tqdm
from pathlib import Path
import tsfel
import numpy as np
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import (accuracy_score, f1_score, log_loss, confusion_matrix, classification_report)
from sklearn.base import clone
import numpy as np
from tabulate import tabulate
import lightgbm as lgb
from sklearn.utils.class_weight import compute_class_weight


## Utilidades

In [2]:
def check_robustness(metric_name, metric_values, threshold_percent=10):
    """
    Check the robustness of a model based on k-fold metric values.
    
    A model is considered robust if the coefficient of variation (CV),
    defined as (standard deviation / mean) * 100, is less than threshold_percent.
    
    Parameters:
        metric_values (list or np.array): List/array of metric values from each fold.
        threshold_percent (float): The maximum allowed CV percentage (default: 10).
        
    Returns:
        bool: True if the model is robust (CV < threshold_percent), False otherwise.
    """
    metric_values = np.array(metric_values)
    mean_val = np.mean(metric_values)
    std_val = np.std(metric_values)
    # Avoid division by zero:
    if mean_val == 0:
        cv = float('inf')
    else:
        cv = (std_val / mean_val) * 100

    print(f"[{metric_name}] Metric Mean: {mean_val:.4f}")
    print(f"[{metric_name}] Metric Standard Deviation: {std_val:.4f}")
    print(f"[{metric_name}] Coefficient of Variation (CV): {cv:.2f}%")
    
    if cv < threshold_percent:
        print(f"[{metric_name}] Model is robust (CV < {threshold_percent}%)")
        return True
    else:
        print(f"[{metric_name}] Model is not robust (CV >= {threshold_percent}%)")
        return False
    
def load_movement_features(subject_id, base_path='../../data/preprocessed/movement/', sampling_rate=100):
    """
    Load and structure movement data from binary files with proper validation
    
    Args:
        subject_id (str): Subject identifier (e.g., '001')
        base_path (str): Base directory for movement data
        sampling_rate (int): Sampling rate in Hz (used for validation)
    
    Returns:
        dict: Structured data with keys as channel names and values as time series arrays
    """
    # File path handling
    path = Path(base_path) / f'{subject_id}_ml.bin'
    if not path.exists():
        raise FileNotFoundError(f"Movement data not found for subject {subject_id}")
    
    # Load raw data
    raw_data = np.fromfile(path, dtype=np.float32)
    
    # Channel configuration (from your specification)
    tasks = ["Relaxed1", "Relaxed2", "RelaxedTask1", "RelaxedTask2", 
             "StretchHold", "HoldWeight", "DrinkGlas", "CrossArms", 
             "TouchNose", "Entrainment1", "Entrainment2"]
    wrists = ["Left", "Right"]
    sensors = ["Accelerometer", "Gyroscope"]
    axes = ["X", "Y", "Z"]
    
    # Calculate expected parameters
    n_channels = len(tasks) * len(wrists) * len(sensors) * len(axes)
    # Even though the duration was 10.24 per assessment, the first ~0.5 seconds are not used and thus
    # the preprocessed data has only 9.76 seconds of data
    expected_duration = 9.76  # seconds per assessment
    expected_timepoints = int(expected_duration * sampling_rate)  # 976
    
    # Validate data size
    expected_size = n_channels * expected_timepoints
    if len(raw_data) != expected_size:
        raise ValueError(f"Unexpected data size for {subject_id}: "
                         f"Got {len(raw_data)} elements, expected {expected_size}")
    
    # Reshape and structure the data
    structured_data = {}
    channel_idx = 0
    
    for task in tasks:
        for wrist in wrists:
            for sensor in sensors:
                for axis in axes:
                    # Extract channel data
                    start = channel_idx * expected_timepoints
                    end = (channel_idx + 1) * expected_timepoints
                    
                    # Create channel name
                    channel_name = f"{task}_{wrist}_{sensor.split(' ')[0]}_{axis}"
                    
                    structured_data[channel_name] = raw_data[start:end]
                    
                    channel_idx += 1

    return structured_data

def extract_tsfel_ts_features(channel_data, domain=None, fs=100):
    """
    Extract TSFEL features from a single-channel time series.
    
    Args:
        channel_data (np.ndarray): 1D array of time series values.
        domain: (str): Domain of features to extract (default: 'all').
            - 'statistical', 'temporal', 'spectral', 'fractal': Includes the corresponding feature domain.
            - 'all': Includes all available feature domains.
            - list of str: A combination of the above strings, e.g., ['statistical', 'temporal'].
            - None: By default, includes the 'statistical', 'temporal', and 'spectral' domains.
        fs (int): Sampling frequency (default: 100).

    Returns:
        np.ndarray: Array of TSFEL features.
    """
    # Obtain a default configuration covering all domains.
    cfg = tsfel.get_features_by_domain(domain)

    # Extract features; the result is a DataFrame with one row.
    features_df = tsfel.time_series_features_extractor(cfg, channel_data, fs=fs, verbose=0)
    # Flatten to 1D numpy array and return.
    return features_df.values.flatten()

def extract_ts_features(label, channel_data, domain=None, fs=None):
    if label == 'basic':
        features = [
            np.mean(channel_data),
            np.std(channel_data),
            np.min(channel_data),
            np.max(channel_data),
            np.percentile(channel_data, 25),
            np.percentile(channel_data, 75),
            np.var(channel_data),
            len(np.where(np.diff(np.sign(channel_data)))[0]) / len(channel_data),  # Zero-crossing rate
            np.sqrt(np.mean(channel_data**2))  # Root Mean Square (RMS)
        ]
        return features
    else:
        return extract_tsfel_ts_features(channel_data, domain=domain, fs=fs)


## Carga de datos

Requisite: Python 3.12 (por catboost). brew install libomp para xgboost

En base a los datos preprocesados, entrenaremos modelos de clasificacion bajo distintos escenarios:


In [8]:
# Cargar metadatos
file_list = pd.read_csv('../../data/preprocessed/file_list.csv',  dtype={'id': str})

movement_data_per_subject = {}
sampling_rate = 100

# For each channel, extract TSFEL features and overwrite the original data

# Create multiple cases so we can check each model independently:
# 1. Basic statistical features
# 2. TSFEL temporal, statistical, and spectral features
# 3. TSFEL only temporal features
# 4. TSFEL only statistical features
# 5. TSFEL only spectral features
# 6. Only questionnaire data

# pipeline_labels = ['basic', 'TSFEL-all', 'TSFEL-temporal', 'TSFEL-statistical', 'TSFEL-spectral', 'questionnaire-only']
pipeline_labels = ['basic', 'TSFEL-spectral', 'questionnaire-only']

pipeline_args = [
    {'domain': None, 'fs': None},  # Basic statistical features
    #{'domain': None, 'fs': sampling_rate},  # TSFEL-all
    #{'domain': 'temporal', 'fs': sampling_rate},  # TSFEL-temporal
    #{'domain': 'statistical', 'fs': sampling_rate},  # TSFEL-statistical
    {'domain': 'spectral', 'fs': sampling_rate},  # TSFEL-spectral
    None,  # Questionnaire-only
]

# X must be 2D (samples, features)
X_vals = {label: [] for label in pipeline_labels}
y_vals = {label: [] for label in pipeline_labels}

# First, load timeseries data and extract features
for _, row in tqdm(file_list.iterrows(), total=len(file_list)):
    # Cargar cuestionario
    quest_data = np.fromfile(f'../../data/preprocessed/questionnaire/{row["id"]}_ml.bin', 
                           dtype=np.float32)
    
    # Load structured movement data
    movement_data = load_movement_features(row["id"])

    """
    movement_data:
    {
        'Relaxed1_Left_Accelerometer_X': np.array([...]),
        'Relaxed1_Left_Accelerometer_Y': np.array([...]),
        ...
    }
    Each key corresponds to a channel, and the value is a 1D numpy array of time series data.
    We have a total of 132 keys and each time series has 976 time points.
    """


    for label, args in zip(pipeline_labels, pipeline_args):
        # Extract features for each channel unless label is 'questionnaire-only'
        features = quest_data
        ts_features = {}
        if label != 'questionnaire-only':
            for channel_name, channel_data in movement_data.items():
                # Extract features using the specified domain and sampling rate
                ts_features[channel_name] = extract_ts_features(label, channel_data, domain=args['domain'], fs=args['fs'])

            # Concatenate features from all channels
            concat_ts_features = np.concatenate(list(ts_features.values()), axis=0)

            # Combine questionnaire data with time series features
            features = np.concatenate((features, concat_ts_features), axis=0)
        
        X_vals[label].append(features.reshape(1, -1))
        y_vals[label].append(row['label'])


# Combine all samples for each pipeline
for label in pipeline_labels:
    if X_vals[label]:
        X_vals[label] = np.concatenate(X_vals[label], axis=0)
        y_vals[label] = np.array(y_vals[label])
    else:
        X_vals[label] = np.array([])
        y_vals[label] = np.array([])

for x in X_vals:
    print(f"{x} shape: {X_vals[x].shape}")

 22%|██▏       | 102/469 [02:30<09:02,  1.48s/it]
/Users/santiago/santi/uoc/TFG/pads-tfg/.venv/lib/python3.12/site-packages/tsfel/feature_extraction/calc_features.py:182: SyntaxWarning: invalid escape sequence '\*'
  """Extraction of time series features.


KeyboardInterrupt: 

## Modelos de clasificacion

In [4]:
# -----------------------------------------------------------------------------
# Labels mapping for reference:
# 0: HC
# 1: PD
# 2: DD
# -----------------------------------------------------------------------------

def tune_models(clf, param_grids, X_inner, y_inner, model_name, fit_params=None):
    """
    Tune hyperparameters for each model using GridSearchCV.
    
    Parameters:
        clf (sklearn classifier): Classifier instance to tune.
        param_grids (dict): Dictionary mapping model names to their hyperparameter grids.
        X_inner (np.array or pd.DataFrame): Feature matrix for inner CV.
        y_inner (np.array): Labels for inner CV.
        model_name (str): Name of the model being tuned.
        fit_params (dict): Additional parameters for fitting the model.
        
    Returns:
        tuned_results (dict): Dictionary mapping model names to the fitted GridSearchCV object.
    """
    # Use GridSearchCV with our custom refit_strategy.
    scoring = {"accuracy": "accuracy", "f1_weighted": "f1_weighted"}
    grid = GridSearchCV(
        estimator=clf,
        param_grid=param_grids[model_name],
        scoring=scoring,
        cv=3,       # inner CV folds
        n_jobs=-1,
        refit=refit_strategy  # custom refit to pick robust candidate
    )
    grid.fit(X_inner, y_inner, **fit_params)
    return (grid.best_estimator_, grid.best_params_)

def refit_strategy(cv_results):
    """
    Custom refit strategy that uses both accuracy and weighted F1 metrics.
    
    For each hyperparameter candidate (as provided in GridSearchCV’s cv_results_),
    we compute the mean and standard deviation for both 'accuracy' and 'f1_weighted'.
    We then calculate the coefficient of variation (CV) for each metric in percentage.
    The composite score is calculated as:
    
        composite = 0.5*(mean_accuracy + mean_f1_weighted) - λ * 0.5*(CV_accuracy + CV_f1_weighted)
    
    A candidate with a high mean score and a low CV will be preferred.
    
    Parameters:
        cv_results (dict): The cv_results_ dictionary from GridSearchCV. It must contain the keys:
            "mean_test_accuracy", "std_test_accuracy", 
            "mean_test_f1_weighted", "std_test_f1_weighted".
    
    Returns:
        best_index (int): The index (as in cv_results) of the best candidate.
    """
    df = pd.DataFrame(cv_results)
    required_keys = ["mean_test_accuracy", "std_test_accuracy", "mean_test_f1_weighted", "std_test_f1_weighted"]
    if not all(key in df.columns for key in required_keys):
        raise ValueError(f"cv_results must contain the keys: {required_keys}")
    
    # Means and standard deviations for accuracy and weighted F1.
    mean_acc = df["mean_test_accuracy"]
    std_acc  = df["std_test_accuracy"]
    mean_f1  = df["mean_test_f1_weighted"]
    std_f1   = df["std_test_f1_weighted"]
    
    # Compute coefficient of variation (CV) in percentages.
    cv_acc = (std_acc / mean_acc) * 100
    cv_f1  = (std_f1 / mean_f1) * 100
    
    # Lambda controls the weight given to robustness. Adjust as needed.
    lambda_val = 0.01
    
    # Composite score: we want high mean and low CV.
    composite = 0.5 * (mean_acc + mean_f1) - lambda_val * 0.5 * (cv_acc + cv_f1)

    best_index = composite.idxmax()
    return best_index

def run_cv(X, y, models, n_splits=5, mode="default", class_weights=None, tune_inner=False, param_grids=None):
    """
    Unified cross-validation runner with optional inner-loop hyperparameter tuning.
    
    Parameters:
        X (np.array or pd.DataFrame): Feature matrix.
        y (np.array): Labels.
        models (dict): Dictionary mapping model names to a tuple: (classifier instance, fit_params dict).
                       (If no additional fit parameters are needed, use an empty dict.)
        n_splits (int): Number of outer CV folds.
        mode (str): One of:
            - "default": standard CV.
            - "smote": apply SMOTE oversampling on the training data.
            - "weighted": for multi-class cost-sensitive learning.
            - "weighted_binary": for binary cost-sensitive learning (with special handling for XGBoost).
        class_weights (dict): Custom class weighting dictionary (e.g., {0: 1.0, 1: 2.0}).
        tune_inner (bool): If True and param_grids is provided, perform inner-loop GridSearchCV tuning.
        param_grids (dict): Dictionary mapping model names to their hyperparameter grids for tuning.
                           Only used if tune_inner is True.
        
    Returns:
        dict: For each model, a dictionary with keys:
            "accuracy": list of accuracies per outer fold,
            "f1_score": list of weighted F1 scores per outer fold,
            "log_loss": list of log losses per outer fold,
            "y_true": list of true label arrays per fold,
            "y_pred": list of predicted label arrays per fold,
            "y_pred_proba": list of predicted probability arrays per fold.
    """
    # Suppress warnings regarding use_label_encoder and feature names
    # Ensure X is a DataFrame with valid feature names.
    if not hasattr(X, "columns"):
        X = pd.DataFrame(X, columns=[f"f{i}" for i in range(X.shape[1])])

    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    # Store results as an array of "folds" for each model.
    results = { name: [] for name in models.keys() }

    # Initialize SMOTE if selected.
    if mode == "smote":
        from imblearn.over_sampling import SMOTE
        oversampler = SMOTE(random_state=42)

    # Outer CV loop.
    for train_idx, test_idx in skf.split(X, y):
        X_train, X_test = X.iloc[train_idx].copy(), X.iloc[test_idx].copy()
        y_train, y_test = y[train_idx], y[test_idx]

        # Optionally apply SMOTE.
        if mode == "smote":
            X_train, y_train = oversampler.fit_resample(X_train, y_train)

        # For each model:
        for name, (model, fit_params) in models.items():
            clf = clone(model)
            fold_fit_params = fit_params.copy() if fit_params is not None else {}

            # For weighted modes, set class_weight if supported.
            if mode in ("weighted", "weighted_binary") and class_weights is not None:
                if 'class_weight' in clf.get_params():
                    clf.set_params(class_weight=class_weights)

            # Special handling for XGBoost in weighted_binary mode.
            if mode == "weighted_binary":
                try:
                    from xgboost import XGBClassifier
                    if name == "XGBoost" and isinstance(clf, XGBClassifier) and len(np.unique(y_train)) == 2:
                        ratio = class_weights.get(1, 1.0) / class_weights.get(0, 1.0)
                        clf.set_params(objective='binary:logistic', scale_pos_weight=ratio)
                except ImportError:
                    pass

            # For LightGBM: enforce early stopping parameters if not provided and set verbosity to -1.
            if name == "LightGBM":
                if "eval_set" not in fold_fit_params:
                    fold_fit_params["eval_set"] = [(X_test, y_test)]
                if "eval_metric" not in fold_fit_params:
                    fold_fit_params["eval_metric"] = "logloss"
                clf.set_params(verbose=-1)

            # --- Inner-loop: Hyperparameter tuning ---
            hyperparameter_tuning_best_params = None
            if tune_inner and param_grids is not None and name in param_grids:
                # Perform hyperparameter tuning using GridSearchCV.
                print(f"Tuning hyperparameters for {name}...")
                (best_estimator, best_params) = tune_models(clf, param_grids, X_train, y_train, name, fit_params=fold_fit_params)
                clf = best_estimator
                hyperparameter_tuning_best_params = best_params
            else:
                clf.fit(X_train, y_train, **fold_fit_params)

            # Evaluate on the outer test set.
            y_pred = clf.predict(X_test)
            y_pred_proba = clf.predict_proba(X_test)

            # Store each model's results in the "results" array, where each outer fold is indexed by k_idx.
            # And each element in the results array is a metrics dictionary.
            model_metrics = {}
            
            model_metrics["params"] = hyperparameter_tuning_best_params if hyperparameter_tuning_best_params else "default",
            model_metrics["accuracy"] = accuracy_score(y_test, y_pred)
            model_metrics["f1_score"] = f1_score(y_test, y_pred, average='weighted')
            model_metrics["log_loss"] = log_loss(y_test, y_pred_proba)
            model_metrics["y_true"] = y_test
            model_metrics["y_pred"] = y_pred
            model_metrics["y_pred_proba"] = y_pred_proba

            print(f"Model results for {name}:")
            print(f"Parameters: {model_metrics['params']}")
            print(f"Accuracy: {model_metrics['accuracy']:.4f}")
            print(f"F1 Score: {model_metrics['f1_score']:.4f}")
            print(f"Log Loss: {model_metrics['log_loss']:.4f}")
            
            results[name].append(model_metrics)

    # Check robustness for each model (using your check_robustness function)
    for name in results.keys():
        # The metrics stored for each model are in an array of dictionaries, where each dictionary corresponds to a fold.
        # We need to extract the metrics from each fold and check their robustness.
        acc_values = [results[name][k]["accuracy"] for k in range(n_splits)]
        f1_values = [results[name][k]["f1_score"] for k in range(n_splits)]
        ll_values = [results[name][k]["log_loss"] for k in range(n_splits)]

        print(f"\nRobustness for model: {name}")
        acc_robust = check_robustness("accuracy", acc_values)
        f1_robust = check_robustness("f1_score", f1_values)
        ll_robust = check_robustness("log_loss", ll_values)
        if acc_robust and f1_robust and ll_robust:
            print(f"{name} is robust across folds.\n")
        else:
            print(f"[ERROR] {name} is not robust across folds.\n")

    return results

def evaluate_cv(results, target_names):
    """
    Aggregates per-fold metrics from a results dictionary and prints overall metrics,
    confusion matrix, and classification report using tabulate.
    
    Parameters:
        results (dict): Dictionary mapping model names to a list of per-fold metric dictionaries.
                        Each per-fold dictionary must include:
                            "accuracy": float,
                            "f1_score": float,
                            "log_loss": float,
                            "y_true": true labels for the fold,
                            "y_pred": predicted labels for the fold,
                            "y_pred_proba": predicted probability array for the fold,
                            "params": the model parameters used.
        target_names (list): List of class names (e.g., ['HC', 'PD', 'DD']).
    
    Returns:
        dict: Mapping of model names to overall metrics including:
            "accuracy_mean", "accuracy_std", "f1_mean", "f1_std", 
            "log_loss_mean", "log_loss_std", "confusion_matrix", and "classification_report".
    """
    from sklearn.metrics import confusion_matrix, classification_report
    import numpy as np
    from tabulate import tabulate

    overall_metrics = {}
    table_data = []

    # From target_names, print what we are evaluating
    print(f"Evaluating models with target names: {target_names}")
    print(f"Number of classes: {len(target_names)}")
    print(f"Classes: {target_names}")
    print(f"Number of models: {len(results)}")
    print(f"Models: {list(results.keys())}")

    for name, fold_results in results.items():
        # Aggregate per-fold predictions.
        y_true_all = np.concatenate([fold["y_true"] for fold in fold_results])
        y_pred_all = np.concatenate([fold["y_pred"] for fold in fold_results])
        y_pred_proba_all = np.concatenate([fold["y_pred_proba"] for fold in fold_results])
        
        # Aggregate per-fold metric values.
        acc_values = np.array([fold["accuracy"] for fold in fold_results])
        f1_values  = np.array([fold["f1_score"] for fold in fold_results])
        ll_values  = np.array([fold["log_loss"] for fold in fold_results])
        
        # Retrieve parameters (assumed constant across folds).
        params_val = fold_results[0].get("params", "default")
        if isinstance(params_val, tuple) and len(params_val) == 1:
            params_val = params_val[0]
        
        # Compute mean and standard deviation for numeric metrics.
        acc_mean, acc_std = np.mean(acc_values), np.std(acc_values)
        f1_mean,  f1_std  = np.mean(f1_values),  np.std(f1_values)
        ll_mean,  ll_std  = np.mean(ll_values),  np.std(ll_values)
        
        # Compute the overall confusion matrix and classification report.
        cm = confusion_matrix(y_true_all, y_pred_all)
        clf_report = classification_report(y_true_all, y_pred_all, target_names=target_names, zero_division=0)
        
        overall_metrics[name] = {
            "params": params_val,
            "accuracy_mean": acc_mean,
            "accuracy_std": acc_std,
            "f1_mean": f1_mean,
            "f1_std": f1_std,
            "log_loss_mean": ll_mean,
            "log_loss_std": ll_std,
            "confusion_matrix": cm,
            "classification_report": clf_report,
        }
        table_data.append([
            name,
            params_val,
            f"{acc_mean:.4f} ± {acc_std:.4f}",
            f"{f1_mean:.4f} ± {f1_std:.4f}",
            f"{ll_mean:.4f} ± {ll_std:.4f}"
        ])
        
        # Print detailed report for this model.
        print(f"Model: {name}")
        print(f"Parameters: {params_val}")
        print("Confusion Matrix:")
        print(cm)
        print("\nClassification Report:")
        print(clf_report)
        print("\n")
    
    print("Overall Metrics:")
    print(tabulate(table_data, headers=["Model", "Params", "Accuracy", "Weighted F1", "Log Loss"], tablefmt="pipe"))
    
    return overall_metrics

param_grids = {
    "RandomForest": {
        "n_estimators": [100, 300],#, 500],
        "max_depth": [None, 10],#, 20],
        "max_features": ["sqrt", "log2"]
    },
    "XGBoost": {
        "learning_rate": [0.01, 0.05],#, 0.1, 0.3],
        "max_depth": [3, 6],#, 9, 12],
        "subsample": [0.5, 0.7],#, 1.0]
    },
    "CatBoost": {
        "depth": [4, 6],#, 8, 10],
        "learning_rate": [0.01],#, 0.03, 0.1],
        "l2_leaf_reg": [1, 3],#, 5, 10]
    },
    "LightGBM": {
        "num_leaves": [20, 31],#, 50, 100],
        "learning_rate": [0.01, 0.05],#, 0.1, 0.2],
        "min_child_samples": [10, 20],#, 30, 50]
    }
}

models = {
     "RandomForest": [RandomForestClassifier(random_state=42), {}],
     "XGBoost": [XGBClassifier(eval_metric='mlogloss', random_state=42), {}],
     "CatBoost": [CatBoostClassifier(verbose=0, random_state=42), {}],
     "LightGBM": [LGBMClassifier(random_state=42), {'callbacks': [lgb.early_stopping(10, verbose=0), lgb.log_evaluation(period=0)]}],
#     "LightGBM": [LGBMClassifier(random_state=42), {}]

}

## Modelos multiclase (PD vs DD vs HC)

In [5]:
for x in X_vals:
    X = X_vals[x]
    y = y_vals[x]
    print(f"Running models for {x} pipeline...")

    # -----------------------------------------------------------------------------
    # Multi-Class Classification (no changes): PD vs DD vs HC
    # -----------------------------------------------------------------------------
    print(len(np.unique(y)))
    if len(np.unique(y)) == 3:
        print("Performing multi-class classification (PD vs DD vs HC) [Default Mode]...")
        results_default = run_cv(X, y, models, n_splits=5, mode="default", tune_inner=True, param_grids=param_grids)
        overall_default = evaluate_cv(results_default, target_names=['HC', 'PD', 'DD'])
    else:
        print("Multi-class classification (PD vs DD vs HC) is not possible with the current labels.")


    # -----------------------------------------------------------------------------
    # Multi-Class Classification with SMOTE: PD vs DD vs HC
    # -----------------------------------------------------------------------------
    if len(np.unique(y)) == 3:
        print("Performing multi-class classification (PD vs DD vs HC) with SMOTE...")
        results_smote = run_cv(X, y, models, n_splits=5, mode="smote", tune_inner=True, param_grids=param_grids)
        overall_smote = evaluate_cv(results_smote, target_names=['HC', 'PD', 'DD'])
    else:
        print("Multi-class classification with SMOTE is not possible with the current labels.")

    # -----------------------------------------------------------------------------
    # Multi-Class Classification with Cost-Sensitive Learning (PD vs DD vs HC)
    # -----------------------------------------------------------------------------
    if len(np.unique(y)) == 3:
        print("Performing multi-class classification (PD vs DD vs HC) with cost-sensitive learning...")
        classes = np.unique(y)
        weights = compute_class_weight(class_weight='balanced', classes=classes, y=y)
        class_weights_multi = dict(zip(classes, weights))
        print("Computed class weights:", class_weights_multi)
        
        results_weighted = run_cv(
            X, y,
            models,
            n_splits=5,
            mode="weighted",
            class_weights=class_weights_multi,
            tune_inner=True,
            param_grids=param_grids
        )
        overall_weighted = evaluate_cv(results_weighted, target_names=['HC', 'PD', 'DD'])
    else:
        print("Multi-class classification with cost-sensitive learning is not possible with the current labels.")

3
Performing multi-class classification (PD vs DD vs HC) [Default Mode]...
Tuning hyperparameters for RandomForest...
Model results for RandomForest:
Parameters: ({'max_depth': None, 'max_features': 'sqrt', 'n_estimators': 100},)
Accuracy: 0.6596
F1 Score: 0.5988
Log Loss: 0.7920
Tuning hyperparameters for XGBoost...
Model results for XGBoost:
Parameters: ({'learning_rate': 0.01, 'max_depth': 3, 'subsample': 0.7},)
Accuracy: 0.6809
F1 Score: 0.6162
Log Loss: 0.8610
Tuning hyperparameters for CatBoost...
Model results for CatBoost:
Parameters: ({'depth': 4, 'l2_leaf_reg': 1, 'learning_rate': 0.01},)
Accuracy: 0.6809
F1 Score: 0.6318
Log Loss: 0.7582
Tuning hyperparameters for LightGBM...
Model results for LightGBM:
Parameters: ({'learning_rate': 0.01, 'min_child_samples': 10, 'num_leaves': 20},)
Accuracy: 0.6489
F1 Score: 0.5637
Log Loss: 0.7821
Tuning hyperparameters for RandomForest...
Model results for RandomForest:
Parameters: ({'max_depth': 10, 'max_features': 'sqrt', 'n_estimators

## Modelos Binarios (PD vs DD) / (PD vs HC)

In [6]:
# -----------------------------------------------------------------------------
# Labels mapping for reference:
# 0: HC
# 1: PD
# 2: DD
# -----------------------------------------------------------------------------

# -------------------------------
# Binary Classification: PD vs DD (Default Mode)
# -------------------------------
if set([1, 2]).issubset(set(y)):
    print("\nPerforming binary classification: PD vs DD [Default Mode]...")
    mask_pd_dd = np.isin(y, [1, 2])
    X_pd_dd = X[mask_pd_dd]
    y_pd_dd = y[mask_pd_dd]
    
    # Adjust labels: PD (1) becomes 0, DD (2) becomes 1
    y_pd_dd_binary = y_pd_dd - 1
    
    results_pd_dd_default = run_cv(X_pd_dd, y_pd_dd_binary, models, n_splits=5, mode="default", tune_inner=True, param_grids=param_grids)
    overall_pd_dd_default = evaluate_cv(results_pd_dd_default, target_names=['PD', 'DD'])
else:
    print("\nBinary classification (PD vs DD) is not possible with the current labels.")

# -------------------------------
# Binary Classification: PD vs HC (Default Mode)
# -------------------------------
if set([1, 0]).issubset(set(y)):
    print("\nPerforming binary classification: PD vs HC [Default Mode]...")
    mask_pd_hc = np.isin(y, [1, 0])
    X_pd_hc = X[mask_pd_hc]
    y_pd_hc = y[mask_pd_hc]
    
    results_pd_hc_default = run_cv(X_pd_hc, y_pd_hc, models, n_splits=5, mode="default", tune_inner=True, param_grids=param_grids)
    overall_pd_hc_default = evaluate_cv(results_pd_hc_default, target_names=['HC', 'PD'])
else:
    print("\nBinary classification (PD vs HC) is not possible with the current labels.")

# -------------------------------
# Binary Classification: PD vs DD with SMOTE
# -------------------------------
if set([1, 2]).issubset(set(y)):
    print("\nPerforming binary classification: PD vs DD with SMOTE...")
    mask_pd_dd = np.isin(y, [1, 2])
    X_pd_dd = X[mask_pd_dd]
    y_pd_dd = y[mask_pd_dd]
    
    # Adjust labels: PD (1) becomes 0, DD (2) becomes 1
    y_pd_dd_binary = y_pd_dd - 1
    
    results_pd_dd_smote = run_cv(X_pd_dd, y_pd_dd_binary, models, n_splits=5, mode="smote", tune_inner=True, param_grids=param_grids)
    overall_pd_dd_smote = evaluate_cv(results_pd_dd_smote, target_names=['PD', 'DD'])
else:
    print("Binary classification (PD vs DD) is not possible with the current labels.")

# -------------------------------
# Binary Classification: PD vs HC with SMOTE
# -------------------------------
if set([1, 0]).issubset(set(y)):
    print("\nPerforming binary classification: PD vs HC with SMOTE...")
    mask_pd_hc = np.isin(y, [1, 0])
    X_pd_hc = X[mask_pd_hc]
    y_pd_hc = y[mask_pd_hc]
    
    results_pd_hc_smote = run_cv(X_pd_hc, y_pd_hc, models, n_splits=5, mode="smote", tune_inner=True, param_grids=param_grids)
    overall_pd_hc_smote = evaluate_cv(results_pd_hc_smote, target_names=['HC', 'PD'])
else:
    print("Binary classification (PD vs HC) is not possible with the current labels.")

# -------------------------------
# Binary Classification: PD vs DD with Cost-Sensitive Learning (Weighted Binary Mode)
# -------------------------------
if set([1, 2]).issubset(set(y)):
    print("\nPerforming binary classification: PD vs DD with cost-sensitive learning...")
    mask_pd_dd = np.isin(y, [1, 2])
    X_pd_dd = X[mask_pd_dd]
    y_pd_dd = y[mask_pd_dd]
    
    # Adjust labels: PD (1) becomes 0 and DD (2) becomes 1.
    y_pd_dd_binary = y_pd_dd - 1

    # Compute balanced class weights from the original y_pd_dd
    classes = np.unique(y_pd_dd)  # Typically [1, 2]
    weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_pd_dd)
    # Create a dictionary mapping original classes to their weight.
    class_weights_pd_dd = dict(zip(classes, weights))
    # Remap keys to reflect the new labels: PD becomes 0, DD becomes 1.
    class_weights_pd_dd = {k - 1: v for k, v in class_weights_pd_dd.items()}
    print("Computed class weights for PD vs DD:", class_weights_pd_dd)
    
    results_pd_dd_weighted = run_cv(
        X_pd_dd,
        y_pd_dd_binary,
        models,
        n_splits=5,
        mode="weighted_binary",
        class_weights=class_weights_pd_dd,
        tune_inner=True,
        param_grids=param_grids
    )
    overall_pd_dd_weighted = evaluate_cv(results_pd_dd_weighted, target_names=['PD', 'DD'])
else:
    print("Binary classification (PD vs DD) is not possible with the current labels.")

# -------------------------------
# Binary Classification: PD vs HC with Cost-Sensitive Learning (Weighted Binary Mode)
# -------------------------------
if set([1, 0]).issubset(set(y)):
    print("\nPerforming binary classification: PD vs HC with cost-sensitive learning...")
    mask_pd_hc = np.isin(y, [1, 0])
    X_pd_hc = X[mask_pd_hc]
    y_pd_hc = y[mask_pd_hc]
    
    # Compute balanced class weights from the current training subset.
    # Here, we assume that in PD vs HC, class 0 = HC and class 1 = PD.
    classes = np.unique(y_pd_hc)
    weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_pd_hc)
    class_weights_pd_hc = dict(zip(classes, weights))
    print("Computed class weights:", class_weights_pd_hc)
    
    results_pd_hc_weighted = run_cv(
        X_pd_hc,
        y_pd_hc,
        models,
        n_splits=5,
        mode="weighted_binary",
        class_weights=class_weights_pd_hc,
        tune_inner=True,
        param_grids=param_grids
    )
    overall_pd_hc_weighted = evaluate_cv(results_pd_hc_weighted, target_names=['HC', 'PD'])
else:
    print("Binary classification (PD vs HC) is not possible with the current labels.")



Performing binary classification: PD vs DD [Default Mode]...
Tuning hyperparameters for RandomForest...
Model results for RandomForest:
Parameters: ({'max_depth': 10, 'max_features': 'sqrt', 'n_estimators': 100},)
Accuracy: 0.7564
F1 Score: 0.7213
Log Loss: 0.5173
Tuning hyperparameters for XGBoost...
Model results for XGBoost:
Parameters: ({'learning_rate': 0.01, 'max_depth': 6, 'subsample': 0.7},)
Accuracy: 0.7308
F1 Score: 0.6818
Log Loss: 0.5375
Tuning hyperparameters for CatBoost...
Model results for CatBoost:
Parameters: ({'depth': 6, 'l2_leaf_reg': 3, 'learning_rate': 0.01},)
Accuracy: 0.7179
F1 Score: 0.6722
Log Loss: 0.6083
Tuning hyperparameters for LightGBM...
Model results for LightGBM:
Parameters: ({'learning_rate': 0.05, 'min_child_samples': 10, 'num_leaves': 20},)
Accuracy: 0.7308
F1 Score: 0.7007
Log Loss: 0.5125
Tuning hyperparameters for RandomForest...
Model results for RandomForest:
Parameters: ({'max_depth': None, 'max_features': 'sqrt', 'n_estimators': 100},)
Acc